In [ ]:
!pip install simpletransformers

In [32]:
import torch
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv("data.csv")
df.columns = ['text', 'label']
encoded_dict = {'negative': 0, 'neutral': 1, 'positive': 2}
df['label'] = df.label.map(encoded_dict)

train, test = train_test_split(df, test_size = 0.2)

In [42]:
model = ClassificationModel('bert', 'bert-base-cased', num_labels = 3, 
                            args = {'optimizer':'AdamW', 'reprocess_input_data': True, 'overwrite_output_dir': True}, 
                            use_cuda = True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [43]:
model.train_model(train)

  0%|          | 0/4673 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/585 [00:00<?, ?it/s]

(585, 0.6490730970333785)

In [44]:
result, model_outputs, _ = model.eval_model(test)
result

  0%|          | 0/1169 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/147 [00:00<?, ?it/s]

{'mcc': 0.6658682427006913, 'eval_loss': 0.44872629723581325}

In [45]:
pred = []
for arr in model_outputs:
  pred.append(np.argmax(arr))

In [46]:
true = test['label'].tolist()

In [47]:
accuracy_score(true, pred)

0.806672369546621

In [52]:
def get_result(statement):
  result = model.predict([statement])
  pos = np.where(result[1][0] == np.amax(result[1][0]))
  pos = int(pos[0])
  sentiment_dict = {0:'negative', 1:'neutral', 2:'positive'}
  print(sentiment_dict[pos])
  return

In [60]:
test.sample(10)

,text,label
4635,Strand Associates is expanding its Madison cor...,2
1156,Qualcomm estimated a first-quarter profit betw...,1
5673,"Goldman Sachs, Barclays, HSBC downplay Brexit ...",1
4755,Finnish P+Âyry has been awarded an engineering...,2
1293,$FB still a dog going much lower this week,0
1073,Tim Cockroft brings with him an excellent trac...,2
2404,Active shipping is essential for Finland .,1
4396,"After the transaction , Herttaassa 's holding ...",1
134,"Facebook, near a buy point last week, faces a ...",2
1408,Finnish Kemira 's net sales EUR decreased to E...,0


In [61]:
print(df['text'][4635], "\t", df['label'][4635], "\n")
get_result(df['text'][4635])

Strand Associates is expanding its Madison corporate headquarters with space to add about 100 employees to the engineering firm . 	 2 



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

positive


In [62]:
print(df['text'][2404], "\t", df['label'][2404], "\n")
get_result(df['text'][2404])

Active shipping is essential for Finland . 	 1 



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

neutral


In [63]:
print(df['text'][1293], "\t", df['label'][1293], "\n")
get_result(df['text'][1293])

$FB still a dog going much lower this week 	 0 



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

negative
